In [179]:
import pandas as pd
import advertools
import nltk
from nltk.stem import WordNetLemmatizer 
from collections import Counter

In [55]:
pd.set_option('display.max_colwidth', None)  

In [192]:
data = pd.read_csv('tweets.csv') 

### 1.  Найти самые короткие твиты авторов, author_name которых состоит из 2 или более слов
Понимаю, что можно сделать это меньшим числом строк, но я немного спешила + нужно было убрать эмоджи из ников, чтобы ник "слово + эмоджи" не считался ником из 2 слов.

In [74]:
authors = set(data['author_name'])
two_words = []
for author in authors:
    if isinstance(author, str):
        splitted = author.split()
        just_words = ""
        for word in splitted:
            if word.isalpha():
                just_words = just_words + word + " "
        if len(just_words.strip().split()) >= 2:
            two_words.append(just_words.strip())

for name in two_words:
    name_data = data.loc[data['author_name'] == name]
    tweets = {}
    for item in name_data['tweet_content']:
        try:
            tweets[item] = len(item)
        except:
            pass
    final = sorted(tweets.items(), key=lambda item: item[1])
    try:
        shortest = final[0]
    except:
        pass
    print(name, shortest, '\n')

Carlo Ricardo Prater ('If rolling soy gave it "1 star "...\n\nits probably actually like\n\n5 - 1000000000000000 stars\n\nConservatively.\n\n', 110) 

Katie Murphy ('Today’s etymological surprise-nugget: the word ‘miniature’ comes from the late Latin ‘miniāre’, to colour red with cinnabar (‘minium’), in little manuscript illuminations; it has no connection with words like ‘minimum’ or ‘minor’', 229) 

Bernie Sanders ('The killing of Palestinian demonstrators by Israeli forces in Gaza is tragic. It is the right of all people to protest for a better future without a violent response', 165) 

Lucas Miller ('The internet is the best school ever created. \n\nThe best peers are on the Internet.\nThe best books are on the Internet.  \nThe best teachers are on the Internet. \n\nThe tools for learning are abundant. \nIt’s the desire to learn that’s scarce.\n\n— @nava', 249) 

Patrick Markey ('New research shows that the fear of smart phones and social media was built on a castle made of sand.  

### 2. Частотный список эмоджи

In [193]:
emoji_summary = advertools.extract_emoji([value for value in data['tweet_content'].values if isinstance(value, str)])

In [194]:
emoji_summary['top_emoji']

[('✅', 117),
 ('😂', 101),
 ('👇', 90),
 ('🔥', 52),
 ('🙏', 37),
 ('🚨', 30),
 ('👏', 27),
 ('🇺🇸', 24),
 ('❤️', 22),
 ('⚫', 19),
 ('💯', 17),
 ('👍', 17),
 ('🤔', 17),
 ('🆕', 16),
 ('🚀', 15),
 ('🙌', 15),
 ('🙄', 13),
 ('🤣', 13),
 ('👉', 12),
 ('⚪', 12),
 ('🧠', 10),
 ('😁', 10),
 ('❤', 10),
 ('😀', 9),
 ('👀', 9),
 ('💀', 9),
 ('💰', 8),
 ('➡️', 8),
 ('👋', 8),
 ('⬇', 8),
 ('😆', 8),
 ('🧐', 8),
 ('✨', 8),
 ('💪', 8),
 ('⚡', 8),
 ('💭', 8),
 ('⤵️', 7),
 ('📈', 7),
 ('🌟', 7),
 ('😎', 7),
 ('❗', 7),
 ('🎉', 7),
 ('🤯', 7),
 ('🗣', 7),
 ('▶️', 6),
 ('🌱', 6),
 ('😊', 6),
 ('😍', 6),
 ('⤵', 6),
 ('📚', 6),
 ('👇🏻', 6),
 ('🇯🇵', 6),
 ('✌', 6),
 ('👏🏽', 6),
 ('⭐', 6),
 ('🇨🇦', 5),
 ('👌', 5),
 ('❎', 5),
 ('🎻', 5),
 ('🐲', 5),
 ('😉', 5),
 ('🥁', 5),
 ('🤤', 5),
 ('🔵', 5),
 ('✈️', 5),
 ('♦️', 5),
 ('📖', 4),
 ('🤗', 4),
 ('⬇️', 4),
 ('😭', 4),
 ('😫', 4),
 ('🗝', 4),
 ('🙃', 4),
 ('💜', 4),
 ('😱', 4),
 ('🍷', 4),
 ('📝', 4),
 ('🥩', 4),
 ('💪🏼', 4),
 ('😄', 4),
 ('🥳', 4),
 ('✍️', 4),
 ('🧘\u200d♂️', 4),
 ('🤨', 4),
 ('❌', 4),
 ('😡', 4),
 ('☑️',

### 3. Оценка сентимента всех твитов

In [118]:
with open('positive-words.txt') as file:
    positive = file.read().split()
    
with open('negative-words.txt') as file:
    negative = file.read().split()

In [130]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [132]:
lemmatizer = WordNetLemmatizer()

In [195]:
def get_sentiment(message, pos, neg):
    score = 0
    if isinstance(message, str):
        splitted = nltk.word_tokenize(message)
        lem = [lemmatizer.lemmatize(word) for word in splitted]
        for word in lem:
            if word in positive:
                score += 1
            elif word in negative:
                score -= 1
    return score

In [198]:
data['sentiment_score'] = data['tweet_content'].apply(get_sentiment, args=(positive, negative))

#### Датасет с колонкой sentiment_score

In [199]:
data

,author_name,created_at,handle,likes,retweets,tweet_content,sentiment_score
0,Naval,2019-08-07 22:36:56,naval,7566,1498,"Unresolved thoughts, prematurely pushed out of the mind, pile up in an internal landfill - which eventually pokes out of the subconscious and manifests as chronic, nonspecific anxiety",-2
1,Naval,2019-08-07 05:00:38,naval,21886,5984,"The modern mind is overstimulated and the modern body is understimulated and overfed. \n\nMeditation, exercise, and fasting restore an ancient balance",2
2,Naval,2019-08-07 04:52:33,naval,6462,1266,"The Lindy Effect for startups:\n\nThe longer you go without shipping product, the more likely you will never ship product",0
3,Naval,2019-08-06 08:35:26,naval,466,61,@orangebook_ This was a good tweet.,1
4,Naval,2019-08-06 07:33:20,naval,3971,906,"Social media lowers the cost of raising &amp; joining mobs.\n\nPer the Minority Rule, the most intolerant wins.\n\nMobs replace conversations with extremism. Journalists, politicians, &amp; society follow.\n\nWe can raise the cost up with software to expose and block the perpetually outraged",-1
...,...,...,...,...,...,...,...
31110,Uncanny Insights,2018-12-20 05:28:36,uncannyinsights,3,1,Our general behavior is the reflection of our collective egotism,-1
31111,Uncanny Insights,2018-12-20 05:18:28,uncannyinsights,2,0,In every matter being an unorthodox is a sure way to success,0
31112,Uncanny Insights,2018-12-19 09:50:56,uncannyinsights,31,4,"You can change your way of thinking, by changing your way of thinking you can change your daily routines and thus you can change your fate. Little things make life great",1
31113,Uncanny Insights,2018-12-19 05:49:03,uncannyinsights,5,0,"People fear that they will be dispossessed if they take risks. In real,by taking risks a man changes his fortunes",-2


#### Сортировка датасета по sentiment_score, выбор 5% самых положительных и самых отрицательных твитов

In [152]:
neg_first = data.sort_values('sentiment_score')
pos_first = data.sort_values('sentiment_score', ascending=False)

total = len(data)
percent = (total * 5) // 100

neg_5 = neg_first[:percent]
pos_5 = pos_first[:percent]

### 5% самых положительных твитов

In [201]:
pos_5

,author_name,created_at,handle,likes,retweets,tweet_content,sentiment_score
10518,Zen Black,2018-11-05 06:10:53,z3nblack,8,0,"Truth lies in a bit of this and a bit of that. You need all of them but u have to understand what the right dose is for u. Just being reliant on the mood is not the solution for creative work. Consistency, structure, can lead u to a creative flow if u know how to work with it. <Q> Consistency, like scheduling, is not the optimal way to do creative work.\n\nBest creative work is often done when you are in the mood. It cannot be scheduled in advanced or rationed equally between working days. </Q>",12
519,Naval,2018-08-10 04:39:23,naval,33160,12293,"Doctors won’t make you healthy.\n\nNutritionists won’t make you slim.\n\nTeachers won’t make you smart.\n\nGurus won’t make you calm.\n\nMentors won’t make you rich.\n\nTrainers won’t make you fit.\n\nUltimately, you have to take responsibility. \n\nSave yourself",10
29478,Max Panzer,2019-04-28 20:35:53,becomingaforce,381,91,"Men, DO YOUR DUTY each day...\n\n- maximum effort\n- be honest and truthful\n- be courageous and courteous\n- be honorable and loyal\n- be positive and optimistic\n- be grateful and generous\n- follow your conscience\n\nAnd most importantly...\n\n- kiss your woman ofte",10
7663,Mr. Plenty,2019-08-25 08:36:22,OfWudan,293,55,Women liking rich men doesn’t make them gold diggers. \n\nWomen are attracted to ambitious high value males. This often means rich males. \n\nIt’s GENUINE attraction. Not chasing money. \n\nThat’s like saying girls who like 6packs are “ab diggers”\n\nNo. It’s just attractive to females,9
24374,The Ancient Sage,2018-05-20 06:46:30,TheAncientSage,92,21,"Being kind to others is the most effective way to manipulate them: they feel indebted and obligated to repay your kindness with a greater favor of their own, and as a bonus you secure a great reputation.\n\nThis is why smart people choose kindness if they have the option",9
...,...,...,...,...,...,...,...
6380,Mason 🏃🏻✂️,2019-07-20 14:16:58,webdevMason,312,40,"Courage is probably the highest ROI character trait right now &amp; the most valuable form isn't one most of us have practiced. It's not taking the leap, it's maintaining focus &amp; enthusiasm while you're in freefal",3
31096,Uncanny Insights,2018-12-27 14:57:14,uncannyinsights,18,1,If you give a lot of attention to a work the soul of the work will be gone. Your creative powers will be stopped. So give the mind some relaxation,3
15444,Thomas Sowell,2017-11-20 18:03:06,ThomasSowell,1363,468,"""There are people who go through the motions and people who get the job done. It is amazing how much work you can accomplish just by hiring the latter and firing the former.",3
20778,seneca,2017-12-06 14:28:32,cryptoseneca,1,0,"@moysha_mk just by likes and hashtags, i feel like we hold the same bags 😎 good day to you si",3


### 5% самых отрицательных твитов

In [202]:
neg_5

,author_name,created_at,handle,likes,retweets,tweet_content,sentiment_score
18682,Sahil Lavingia,2019-08-14 20:02:29,shl,11555,2903,"Being a founder is hard. Being an early employee is hard. Being a VC is hard. Being a politician is hard. Being an engineer is hard. Being a writer is hard. Being a friend is hard. Being a partner is hard. Being a parent is hard.\n\nIt's all hard.\n\nLife is hard, and then you die",-12
3243,Wealth Theory ™,2019-05-29 18:28:50,Wealth_Theory,66,11,"When a naive investor who saw no risk suddenly sees risk, he is most vulnerable to panic selling.\n\nWhen a market which saw no risk suddenly sees risk, it is most vulnerable to crash.\n\nZero risk priced in = maximum risk",-11
3365,Wealth Theory ™,2019-05-03 03:20:31,Wealth_Theory,111,19,"When a naive investor who saw no risk suddenly sees risk, he is most vulnerable to panic selling.\n\nWhen a market which saw no risk suddenly sees risk, it is most vulnerable to crash.\n\nZero risk priced in = maximum risk",-11
3626,Wealth Theory ™,2019-02-22 15:38:40,Wealth_Theory,93,16,"When a naive investor who saw no risk suddenly sees risk, he is most vulnerable to panic selling.\n\nWhen a market which saw no risk suddenly sees risk, it is most vulnerable to crash.\n\nZero risk priced in = maximum risk",-11
24316,The Ancient Sage,2018-06-08 01:11:17,TheAncientSage,53,15,"We unconsciously flee from our problems, seeking refuge in mindless, compulsive distractions.\n\nThis doesn't make the problems disappear, though. In fact they worsen.\n\nTheir power lies in our aversion from attending to them. Once we decide to face them, they lose their power",-10
...,...,...,...,...,...,...,...
30109,The Stoic Emperor,2018-03-31 03:30:52,TheStoicEmperor,1177,324,"To see the unfulfilled longings of modern women, read romance novels. \n\nTo see the unfulfilled longings of modern men, play video games.\n\nIntellectuals despise depictions of real human motivation and try to banish it from ""respectable"" cultural institutions.\n\nYet the truth sells",-3
30107,The Stoic Emperor,2018-03-31 16:44:33,TheStoicEmperor,465,135,"We have created a fragile and comfortable modern world in which things are much less likely to go slightly wrong, yet more likely to go catastrophically wrong. \n\nThe future offers up enormous opportunity and enormous existential risk",-3
30106,The Stoic Emperor,2018-03-31 20:14:45,TheStoicEmperor,274,72,Ignorant celebrities are creating a toxic moral environment around vital issues.\n\nCompletely ignorant people are misleading millions of their followers.\n\nIgnore the opinions of those who spend a great deal of time looking in the mirror and none at all looking at the facts,-3
23415,The Wise Bot,2019-09-17 08:00:02,thewisebot,299,62,"""If there's no enemy within, the enemy outside can do little harm."" - @Kpax",-3


In [190]:
def most_freq(data, sent_words):
    result = []
    for value in data['tweet_content'].values:
        splitted = nltk.word_tokenize(value)
        lem = [lemmatizer.lemmatize(word) for word in splitted]
        for word in lem:
            if word in sent_words:
                result.append(word)
    count = Counter(result)
    ten = count.most_common()[:10]    
    return ten

### 10 самых частотных отрицательных слов из отрицательных твитов

In [186]:
neg_words = most_freq(neg_5, negative)

In [187]:
neg_words

[('problem', 198),
 ('fear', 148),
 ('bad', 98),
 ('hard', 89),
 ('risk', 87),
 ('pain', 75),
 ('failure', 74),
 ('hate', 70),
 ('enemy', 69),
 ('mistake', 65)]

### 10 самых частотных положительных слов из положительных твитов

In [191]:
pos_words = most_freq(pos_5, positive)
pos_words

[('like', 327),
 ('work', 319),
 ('good', 256),
 ('better', 178),
 ('right', 163),
 ('love', 163),
 ('best', 158),
 ('great', 132),
 ('enough', 120),
 ('free', 110)]